# Model Deployment with FastAPI

In this notebook, we’ll explore how to deploy a trained Machine Learning model using **FastAPI** : a modern, fast (high-performance) web framework for building APIs with Python.

## 🎯 Objectives
- Understand the basics of FastAPI for ML deployment.
- Serve a trained model via REST API endpoints.
- Test the API using `requests` or a web interface like Swagger UI.
- Package the API for production use.

---

## ⚙️ 1. Setup and Installation

Make sure you have the following dependencies installed:
```bash
pip install fastapi uvicorn scikit-learn joblib
```

We’ll use `uvicorn` as the ASGI server to run our FastAPI app.

## 🧩 2. Load a Sample Model

Let’s create or load a simple trained model (for demonstration, we’ll use a **RandomForestClassifier** on the Iris dataset).

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Train a model
model = RandomForestClassifier()
model.fit(X, y)

# Save the model
joblib.dump(model, 'iris_model.pkl')
print('✅ Model trained and saved as iris_model.pkl')

## 🚀 3. Creating the FastAPI App

Now, we’ll create a simple FastAPI application that:
- Loads the trained model.
- Defines `/predict` endpoint to make predictions.
- Runs the API using `uvicorn`.

In [ ]:
%%writefile app.py

from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

# Load model
model = joblib.load('iris_model.pkl')

# Initialize app
app = FastAPI(title="Iris Classifier API", description="Predict iris species using FastAPI", version="1.0")

# Define input data structure
class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

@app.get("/")
def read_root():
    return {"message": "Welcome to the Iris Classifier API!"}

@app.post("/predict")
def predict(data: IrisInput):
    features = np.array([[data.sepal_length, data.sepal_width, data.petal_length, data.petal_width]])
    prediction = model.predict(features)
    return {"prediction": int(prediction[0])}

## ▶️ 4. Running the API Server

Run this command in your terminal to start the FastAPI server:
```bash
uvicorn app:app --reload
```

Once running, open your browser and navigate to:
- **Swagger UI**: [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)
- **ReDoc UI**: [http://127.0.0.1:8000/redoc](http://127.0.0.1:8000/redoc)

## 🧪 5. Testing the API

You can test the API using Python’s `requests` module or directly from Swagger UI.

In [ ]:
import requests

url = "http://127.0.0.1:8000/predict"
payload = {
    "sepal_length": 5.1,
    "sepal_width": 3.5,
    "petal_length": 1.4,
    "petal_width": 0.2
}

response = requests.post(url, json=payload)
print(response.json())

## 🧱 6. Deploying to Production

### Option 1: Using Docker
Create a `Dockerfile`:
```dockerfile
FROM python:3.10
WORKDIR /app
COPY . /app
RUN pip install fastapi uvicorn scikit-learn joblib
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```
Build and run:
```bash
docker build -t iris-fastapi .
docker run -p 8000:8000 iris-fastapi
```

### Option 2: Using Cloud Platforms
- **Render**, **Railway**, or **AWS EC2** — simply push your app and run using `uvicorn`.
- Add environment management (dotenv) and logging for production readiness.

## ✅ Summary

- You created and trained a simple ML model.
- Deployed it via **FastAPI** as an interactive REST API.
- Tested endpoints using both Swagger UI and `requests`.
- Learned how to containerize it for production use.

---
Next → **06-Continuous_Integration_and_Delivery.ipynb** 📦